In [3]:
!pip install sentence_transformers pinecone-client datasets seaborn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.5 MB/s

In [4]:
# !pip install --upgrade torch torchvision

In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";
os.environ["CUDA_LAUNCH_BLOCKING"]="1";


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,TrainingArguments
)

In [3]:


class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [4]:
def Remove_line(text):
  new=text.replace('\n', ' ')
  return new

In [5]:
# load the dataset and convert to pandas dataframe
df_tain = load_dataset("fhamborg/news_sentiment_newsmtsc",split="train").to_pandas()

In [6]:
df_validation = load_dataset( "fhamborg/news_sentiment_newsmtsc",split="validation").to_pandas()

In [7]:
df_test = load_dataset("fhamborg/news_sentiment_newsmtsc",split="test").to_pandas()

In [8]:
df_tain["sentence"]=df_tain["sentence"].apply(Remove_line)
df_validation["sentence"]=df_validation["sentence"].apply(Remove_line)
df_test["sentence"]=df_test["sentence"].apply(Remove_line)


In [9]:
df_tain["sentence"][14]

'It was a familiar morning for Mr. Trump on his favored medium — and for a nation that had, for a little more than a week, gone without the president’s stream-of-consciousness missives.'

In [10]:
df_tain.polarity.value_counts()

-1    3316
 0    3028
 1    2395
Name: polarity, dtype: int64

In [11]:
df_test["sentence"][0]

'Though we do not know what other items Seth may have had in his possession, his watch, phone, wallet and necklace were not stolen.'

In [12]:
df_test

,mention,polarity,from,to,sentence,id
0,Seth,0,39,43,Though we do not know what other items Seth ma...,polusa_v1_4307505_-1_11_Seth_39_43
1,He,1,0,2,He won the Academy Award for Best Supporting A...,allsides_4276_1784_7_Beloved actor Robin Willi...
2,""" Powell",1,44,52,"""Yup, the whole birther movement was racist,"" ...",allsides_1810_737_6_Powell_44_52
3,Ross,0,0,4,Ross's appointment will require a Senate confi...,allsides_1593_643_5_Wilbur Ross_0_4
4,Andrews,0,27,34,Trustees Simone Boutet and Andrews voted no ea...,polusa_v1_59535720_-1_32_Trustee Deno Andrews_...
...,...,...,...,...,...,...
798,Poe,0,91,94,To have her now say she doesn't support our ri...,allsides_4841_2036_10_Poe_91_94
799,Obama,1,98,103,But some observers suggested it was tone-deaf ...,allsides_2495_1015_21_Obama_98_103
800,Cox,0,95,98,“It is my opinion that you are a key conspirat...,polusa_v1_39150634_-1_5_Sean Cox_95_98
801,Schmidt,-1,104,111,“It is my opinion that you are a key conspirat...,polusa_v1_39150634_-1_5_Schmidt_104_111


In [13]:
# clean_1_test = df_test.groupby('polarity', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
# clean_1_test = clean_1_test.sample(frac=1, random_state=42)
# clean_1_test.dropna(subset=['sentence'], inplace=True)
# clean_1_test

# Read  ALL_datafile

In [14]:
All_data=pd.read_csv("https://raw.githubusercontent.com/AnasElbattra/News-Understanding-NLP/main/Sentiment%20Analysis/Data/all-data.csv", encoding='latin-1', header=None)
All_data

,0,1
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [15]:
All_data = All_data.rename(columns={1: 'sentence', 0: 'polarity'})

In [16]:
All_data.polarity.value_counts()

neutral     2879
positive    1363
negative     604
Name: polarity, dtype: int64

In [17]:
All_data["sentence"]=All_data["sentence"].apply(Remove_line)


In [18]:
All_data["polarity"]=All_data.polarity.map({
    "neutral": 0,
    "positive": 1,
    "negative": -1
})

In [19]:
All_data.polarity.value_counts()

 0    2879
 1    1363
-1     604
Name: polarity, dtype: int64

In [20]:
All_data = All_data.sample(frac=1, random_state=42)

In [21]:
# Splitting the data into training, validation, and test sets
train_All_data, temp = train_test_split(All_data, test_size=0.3, random_state=42, stratify=All_data["polarity"])
val_All_data, test_All_data = train_test_split(temp, test_size=0.5, random_state=42, stratify=temp["polarity"])

In [22]:
train_All_data.shape

(3392, 2)

# Merge the three Dataframe

In [23]:
df_tain_selected=df_tain[["sentence","polarity"]]

train_All_data_selected=train_All_data[["sentence","polarity"]]


In [24]:
# Combine the DataFrames into one using pd.concat
merged_train = pd.concat([df_tain_selected,  train_All_data_selected], ignore_index=True)
merged_train

,sentence,polarity
0,Winner wrote that she had a 30-minute private ...,0
1,She also recently referred to President Trump ...,-1
2,She also recently referred to President Trump ...,-1
3,Hillary Clinton blamed the Democratic National...,-1
4,Hillary Clinton blamed the Democratic National...,-1
...,...,...
12126,"Also , CBA is to issue a benchmark , 10 year f...",0
12127,+_lemiste City is the environment for a knowle...,0
12128,GE is building the facility with wind power de...,0
12129,No financial or pricing details were disclosed .,0


In [25]:
df_validation_selected=df_validation[["sentence","polarity"]]

val_All_data_selected=val_All_data[["sentence","polarity"]]


In [26]:
# Combine the DataFrames into one using pd.concat
merged_val = pd.concat([df_validation_selected,val_All_data_selected], ignore_index=True)
merged_val

,sentence,polarity
0,In the 2015 Conservative Political Action Conf...,0
1,“He was willing to meet with industry to try t...,1
2,"""As far as the truck rental goes, those interv...",0
3,Then Trump didn’t actually let Priebus be a re...,-1
4,Then Trump didn’t actually let Priebus be a re...,-1
...,...,...
1065,"Water Treatment Products In Australia Today , ...",0
1066,Operating profit rose from EUR 1.94 mn to EUR ...,1
1067,"Even so , Tulikivi emissions are well below so...",0
1068,"Yesterday , Legrand issued its E300 million fi...",0


In [27]:
df_test_selected=df_test[["sentence","polarity"]]

test_All_data_selected=test_All_data[["sentence","polarity"]]


In [28]:

merged_test = pd.concat([df_test_selected,  test_All_data_selected], ignore_index=True)
merged_test

,sentence,polarity
0,Though we do not know what other items Seth ma...,0
1,He won the Academy Award for Best Supporting A...,1
2,"""Yup, the whole birther movement was racist,"" ...",1
3,Ross's appointment will require a Senate confi...,0
4,Trustees Simone Boutet and Andrews voted no ea...,0
...,...,...
1525,DMASIA-16 August 2006-Benefon extends manufact...,1
1526,The first group of customers to be trained wil...,0
1527,The bridge will be built to the south of the e...,0
1528,These module products will be available for tr...,0


In [29]:
shuffled_test_df = merged_test.sample(frac=1, random_state=42)
shuffled_val_df = merged_val.sample(frac=1, random_state=42)
shuffled_train_df = merged_train.sample(frac=1, random_state=42)



In [30]:
clean_1_test = shuffled_test_df.groupby('polarity', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
clean_1_test = clean_1_test.sample(frac=1, random_state=42)
clean_1_test.dropna(subset=['sentence'], inplace=True)
clean_1_test

,sentence,polarity
291,"""Even for someone that is that empty a barrel,...",0
205,"In February 2002, Biden told a Delaware audien...",-1
276,"""He must be given his full freedom, like all t...",1
790,One thing is for sure: Wray won’t want to meet...,0
352,He also told the assembly that TfL gave him no...,0
...,...,...
1243,Delivery is due in the second half of 2011 .,0
1370,In addition the deal includes a call option wh...,1
1115,Stora Enso 's target has been cut to EUR 4.85 ...,-1
850,The store is located in Poznan in a shopping c...,0


In [31]:
shuffled_val_df.polarity.value_counts()

 0    568
 1    278
-1    224
Name: polarity, dtype: int64

In [32]:
x_train=train_All_data_selected["sentence"]
y_train=train_All_data_selected["polarity"]

In [33]:
x_valid=shuffled_val_df["sentence"]
y_valid=shuffled_val_df["polarity"]

In [34]:
x_test=clean_1_test["sentence"]
y_test=clean_1_test["polarity"]

In [35]:

# model_id ="lxyuan/distilbert-base-multilingual-cased-sentiments-student"
# model_id ="distilbert-base-uncased"
# model_id ="distilbert-base-uncased-finetuned-sst-2-english"

# model_id ="distilbert-base-uncased-distilled-squad"

# model_id ="Davlan/distilbert-base-multilingual-cased-ner-hrl"
# model_id ="xlnet-base-cased"
# model_id ="t5-small"
# model_id = "cardiffnlp/twitter-roberta-base-sentiment"
model_id ="xlnet-base-cased"
# load the model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=3
                                                          #  , ignore_mismatched_sizes=True
                                                           )
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
train_encoding = tokenizer(x_train.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
valid_encoding = tokenizer(x_valid.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
test_encoding = tokenizer(x_test.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")


In [37]:
y_train = y_train.astype(int)
y_valid = y_valid.astype(int)
y_test = y_test.astype(int)


In [38]:
y_train

4175   -1
3267    0
2798    0
4444   -1
3809    0
       ..
3523    0
2725    0
3604    0
3695    0
3030    0
Name: polarity, Length: 3392, dtype: int64

In [39]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the target labels and transform y_train
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)
y_test_encoded = label_encoder.transform(y_test)



In [40]:
y_train.unique()

array([-1,  0,  1])

In [41]:
{"1":0,"0":-1,"2":1}

{'1': 0, '0': -1, '2': 1}

In [42]:
y_train_encoded

array([0, 1, 1, ..., 1, 1, 1])

In [43]:
train_dataset=IMDbDataset(train_encoding,y_train_encoded)
valid_dataset=IMDbDataset(valid_encoding,y_valid_encoded)
test_dataset=IMDbDataset(test_encoding,y_test_encoded)


In [44]:
train_dataset

In [45]:
training_args = TrainingArguments(
    output_dir='./results',  # Fix the space before the directory path
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=10,
)


In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
    # compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-3-a7e13701bd24>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


OutOfMemoryError: ignored

In [ ]:
# output_dir="saved"
# tokenizer.save_pretrained(output_dir)

# model.save_pretrained(output_dir)

In [ ]:
trainer.save_model()


trainer.save_state()


checkpoint_name = "test-trainer-deepspeed"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"



trainer.save_model(output_dir=final_model_path)

In [ ]:
# fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
# tokenizer_new = AutoTokenizer.from_pretrained(output_dir)


In [ ]:

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(final_model_path)
# tokenizer_new = AutoTokenizer.from_pretrained(final_model_path)

In [ ]:
inputs = tokenizer(x_test.tolist(),return_tensors="pt",padding='max_length', truncation=True, max_length=512)

In [ ]:
import torch.nn.functional as F
with torch.no_grad():
    outputs =fine_tuned_model(**inputs)
    # print (outputs)
    prediction=F.softmax(outputs.logits, dim=1 )
    # print (prediction)

    labels= torch.argmax(prediction, dim=1 )
    # print(labels)
    classification= [fine_tuned_model.config.id2label[label_id] for label_id in labels.tolist()]

In [ ]:
label_mapping = {"LABEL_1": 0, "LABEL_0": -1, "LABEL_2": 1}

classification_nnn= [label_mapping[label_id] for label_id in classification]

In [ ]:
# classification_nnn

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(y_test, classification_nnn))

In [125]:
# !pip install accelerate -U

In [114]:
text="A toddler with a bandaged head, limp and dusty, is carried into a Gaza hospital for treatment after an Israeli airstrike. Mourners bid farewell to a woman who was killed in southern Israel in the Oct.7 Hamas attack on a kibbutz.Scenes of destruction and pain endure on both sides as the latest conflict between Israel and Hamas stretches into a fourth week."

In [115]:
text=[text]

In [116]:
text

['A toddler with a bandaged head, limp and dusty, is carried into a Gaza hospital for treatment after an Israeli airstrike. Mourners bid farewell to a woman who was killed in southern Israel in the Oct.7 Hamas attack on a kibbutz.Scenes of destruction and pain endure on both sides as the latest conflict between Israel and Hamas stretches into a fourth week.']

In [117]:
One_csase = tokenizer(text,return_tensors="pt",padding='max_length', truncation=True, max_length=512)

In [118]:
import torch.nn.functional as F
with torch.no_grad():
    outputs =fine_tuned_model(**One_csase)

    prediction=F.softmax(outputs.logits, dim=1 )


    labels= torch.argmax(prediction, dim=1 )
    print(labels)
    classification= [fine_tuned_model.config.id2label[label_id] for label_id in labels.tolist()]

tensor([1])


In [119]:
label_mapping = {"LABEL_1": 0, "LABEL_0": -1, "LABEL_2": 1}

classification_nnn= [label_mapping[label_id] for label_id in classification]

In [120]:
classification_nnn

[0]

In [121]:
if (classification_nnn[0]==0):
  print ("Neutral")
if (classification_nnn[0]==-1):
  print ("Negative")
if (classification_nnn[0]==1):
  print ("Positive")

Neutral
